## <center>CSE 546: Reinforcement Learning</center>
### <center>Prof. Alina Vereshchaka</center>
<!-- ### <center>Fall 2022</center> -->

Welcome to the Assignment 2, Part 1: Introduction to Deep Reinforcement Learning and Neural Networks! The goal of this assignment is to make you comfortable with the application of different Neural Network structures depending on how the Reinforcement Learning environment is set up.

In [ ]:
# Imports
from environment import WumpusWorldEnvironment
import torch
import torch.nn as nn
import torch.nn.functional as F

# Environment

We will be working with an implementation of the Wumpus World environment. The environment comes from the book "Artificial Intelligence: A Modern Approach" by Stuart J. Russell and Peter Norvig. 

### ENVIRONMENT DETAILS:

The environment is a 6 x 6 grid world containing a total of 36 grid blocks. 

#### ENVIRONMENT OBJECTS:
The environment consists of the following objects:

1. **Agent** - The agent starts in the grid block at the bottom left corner whose co-ordinates are [0, 0]. The goal of our agent is to collect the Gold while avoiding the Wumpus and the pits. 

2. **Wumpus** - The monster which would eat the agent if they are in the same grid block.

3. **Pit** - The agent must avoid falling into the pits. 

4. **Gold** - The agent must collect the Gold.

5. **Breeze** - Breeze surrounds the Pits and warn the agent of a Pit in an adjacent grid block.

6. **Stench** - Stench surrounds the Wumpus and warns the agent of the Wumpus in an adjacent grid block.

#### ENVIRONMENT OBSERVATIONS:

Our implementation of the environment provides you with four different types of observations:

1. **Integer** - Integer in the range [0 - 35]. This represents the grid block the agent is in. E.g., if the agent is in the bottom left grid block (starting position) the observation would be 0, if the agent is in the grid block containing the Gold the observation would be 34, if the agent is in the top right grid block the observation would be 35.

2. **Vector** - 

    **2.1.** A vector of length 2 representing the agent co-ordinates. The first entry represents the x co-ordinate and the second entry represets the y co-ordinate. E.g., if the agent is in the bottom left grid block (starting position) the observation would be [0, 0], if the agent is in the grid block containing the Gold the observation would be [4, 5], if the agent is in the top right grid block the observation would be [5, 5].
    
    **2.2.** A vector of length 36 representing the one-hot encoding of the integer observation (refer type 1 above). E.g., if the agent is in the bottom left grid block (starting position) the observation would be [1, 0, ..., 0, 0], if the agent is in the grid block containing the Gold the observation would be [0, 0, ..., 1, 0], if the agent is in the top right grid block the observation would be [0, 0, ..., 0, 1].


3. **Image** - Image render of the environment returned as an NumPy array. The image size is 84 * 84 (same size used in the DQN paper). E.g., if the agent is in the bottom right grid block the observation is:

    Observation: (84 * 84)

     [[255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     ...

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]]

    Observation type: <class 'numpy.ndarray'>

    Observation Shape: (84, 84)

    Visually, it looks like:
    <img src="./images/environment_render.png" width="500" height="500">
    

4. **Float** - Float in the range [0 - $\infty$] representing the time elapsed in seconds. 

#### ENVIRONMENT ACTIONS:

Our implementation of the environment provides you with three different types of actions:

1. **Discrete** - Integer in the range [0 - 3] representing the four actions possible in the environment as follows: 0 - Right 1 - Left 2 - Up 3 - Down.

2. **Multi-Discrete** - Array of length four where each element takes binary values 0 or 1. Array elements represent if we take a particular action. Array element with index 0 corresponds to the right action, index 1 corresponds to the left action, index 2 corresponds to the up action, and index 3 corresponds to the down action. E.g., 
   action = [1, 0, 0, 0] would result in the agent moving right.
   action = [1, 0, 1, 0] would result in the agent moving right and up.
   action = [0, 1, 0, 1] would result in the agent moving left and down.

3. **Continuous** - Float in the range [-1, 1] determining whether the agent will go left, right, up, or down as follows:

    if -1 <= action <= -0.5:
        Go Right.
    elif -0.5 < action <= 0:
        Go Left.
    elif 0 < action <= 0.5:
        Go Up.
    elif 0.5 < action <= 1:
        Go Down.
        
### YOUR TASK IS TO USE A NEURAL NETWORK TO WORK WITH ALL FOUR TYPES OF OBSERVATIONS AND ALL THREE TYPES OF  ACTIONS.
### Note: You don't have to train your agent/neural network. You just have to build the neural network structure that takes the observation as input and produces the desired output with the initial weights.

#### You can use libraries such as PyTorch/TensorFlow/Keras to build your neural networks.

#### <span style="color:red">You cannot use RL libraries that already provide the neural network to you such as Stable-baselines3, Keras-RL, TF agents, Ray RLLib etc.</span>

<img src="./images/wumpus_world_environment.jpg" width="600" height="600">

# START COMPLETING YOUR ASSIGNMENT HERE

## Observation Type - Integer, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is an integer (refer to environment observations type 1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class IntegerDiscreteNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 4)  #Hidden 64 and 4 output layers

    def forward(self, x):
        x = F.relu(self.fc1(x)) #First layer and then activation
        return self.fc2(x)

model = IntegerDiscreteNN()
observation_tensor = torch.tensor([[observation]], dtype=torch.float32)
q_values = model(observation_tensor) #Getting Q values for all the actionns
print("Observation:", observation)
print("Q-values:", q_values.detach().numpy())


# END_YOUR_CODE

Observation: 0
Q-values: [[-0.19534394 -0.28594893 -0.14653148  0.02260301]]


## Observation Type - Vector (2.1), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 2 (refer to environment observations type 2.1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_2_64_4.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class VectorDiscreteNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 64) #2 neurons and 64 hidden layers
        self.fc2 = nn.Linear(64, 4) #64 hidden and 4 neurons as output layer
    def forward(self, x):
        x = F.relu(self.fc1(x)) #applying reLu after first layer
        return self.fc2(x)


# END_YOUR_CODE

In [5]:
observation_tensor = torch.tensor([observation], dtype=torch.float32)
model = VectorDiscreteNN()
q_values = model(observation_tensor)
print("Observation:", observation)
print("Predicted Q-values:", q_values.detach().numpy())


Observation: [0 0]
Predicted Q-values: [[-0.19492269 -0.19148968 -0.41030824  0.14790136]]


C:\Users\gmhar\AppData\Local\Temp\ipykernel_15908\2310517092.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  observation_tensor = torch.tensor([observation], dtype=torch.float32)


## Observation Type - Vector (2.2), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE

class OneHotDiscreteNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(36, 64) #Input is 36 neurons that one-hot encoding and 64 hidden layers
        self.fc2 = nn.Linear(64, 4) #Hidden 64 layers and 4 output
    def forward(self, x):
        x = F.relu(self.fc1(x)) #ReLu activation 
        return self.fc2(x)

one_hot_obs = F.one_hot(torch.tensor([observation]), num_classes=36).float()
model = OneHotDiscreteNN()
q_values = model(one_hot_obs) #Model and Q-values
print("One-hot observation:", one_hot_obs)
print("Predicted Q-values:", q_values.detach().numpy())

# END_YOUR_CODE

One-hot observation: tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Predicted Q-values: [[-0.00595273  0.03581554 -0.11294824  0.08061178]]


## Observation Type - Image, Action Type - Discrete

The part of the assignment requires you to create a convolutional neural network with one convolutional layer having 128 filters of size 3 x 3, one hidden layer having 64 neurons, and the output layer having 4 neurons. The input to the neural network is an image of size 84 * 84 (refer to environment observations type 3). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/convolutional_neural_network_84x84_128_64_4.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='image', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class ImageDiscreteNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 84 * 84, 64) #fully connected layer. Input is 128 * 84* 84 and 64 output neurons
        self.fc2 = nn.Linear(64, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(x.size(0), -1)      # flattening it to @D
        x = F.relu(self.fc1(x)) #Pass through  hidden layer with ReLu
        return self.fc2(x)


environment = WumpusWorldEnvironment(observation_type='image', action_type='discrete')
obs, info = environment.reset()
obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).unsqueeze(0) 
model = ImageDiscreteNN()
q_values = model(obs_tensor)
print("Q-values:", q_values.detach().numpy())


# END_YOUR_CODE

Q-values: [[ -0.09585664  25.306082   -15.960548     6.82759   ]]


## Observation Type - Float, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a float (refer to environment observations type 4). The output of the neural network is an array representing the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='float', action_type='discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class FloatDiscreteNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 64) # 1 neuron(float) and 64 hidden layer
        self.fc2 = nn.Linear(64, 4) #64 hidden layers and 4 outputs

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)


environment = WumpusWorldEnvironment(observation_type='float', action_type='discrete')
observation, info = environment.reset()


x = torch.tensor([[observation]], dtype=torch.float32)
model = FloatDiscreteNN()
q_values = model(x)

print("Observation:", observation)
print("Predicted Q-values:", q_values.detach().numpy())


# END_YOUR_CODE

Observation: 0.0
Predicted Q-values: [[-0.07450657  0.09674042  0.5840402   0.07710017]]


## Observation Type - Vector (2.2), Action Type - Multi-Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array representing the probability of choosing the actions. (If the value of the array element is >=0.5 you will perform the action.) (refer to environment actions type 2).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4_sigmoid.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action probabilities for the four actions. Print the observation and the action probabilities."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='multi_discrete')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class OneHotMultiDiscreteNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(36, 64)#Inout layer : 36 neurons (one-hot encoding ) and 4 hiddenn
        self.fc2 = nn.Linear(64, 4) #64 hidden and 4 output neuron

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))  


environment = WumpusWorldEnvironment(observation_type='integer', action_type='multi_discrete')
observation, info = environment.reset()
one_hot = F.one_hot(torch.tensor([observation]), num_classes=36).float() #converting int to one-hot encoding

model = OneHotMultiDiscreteNN()
probs = model(one_hot)
print("One-hot observation:", one_hot)
print("Action probabilities:", probs.detach().numpy())


# END_YOUR_CODE

One-hot observation: tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Action probabilities: [[0.47424173 0.50714236 0.52683455 0.49621114]]


## Observation Type - Vector (2.2), Action Type - Continuous

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 1 neuron. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an float in the range [-1, 1] determining the action which will be taken. (refer to environment actions type 3).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector and use the TanH activation function to get the output in the range [-1, 1].

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_1.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action. Print the observation and the predicted action."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='continuous')
observation, info = environment.reset()

# BEGIN_YOUR_CODE
class OneHotContinuousNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(36, 64)
        self.fc2 = nn.Linear(64, 1) #Hidden 64 layer and 1 output layer (continuous action value)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x))   #tanh to get output range [-1,1]


environment = WumpusWorldEnvironment(observation_type='integer', action_type='continuous')
observation, info = environment.reset()
one_hot = F.one_hot(torch.tensor([observation]), num_classes=36).float()

model = OneHotContinuousNN()
action = model(one_hot)
print("Observation:", observation)
print("Predicted continuous action:", action.item())

# END_YOUR_CODE

Observation: 0
Predicted continuous action: -0.09952755272388458
